# Chapter 15 Linear Factor Models and Auto-Encoders
## メタ情報
全部で27ページ。  
DAEの章が2つある、auto-encoderの具体的な数式の記述がない等、整っていないところが多い。  
auto-encodersは今はあまり注目されていないのかなーと思ってしまった。  
だいたい、どの部分にどの分布を仮定するのか、正則化項を設けるのか、の話。  

## はじめに
線形ファクタモデルは観測されない要素$h$で観測変数$x$を線形変換で説明する教師なし学習による生成モデルである。  
auto-encodersも教師なしでデータの表現を学習する手法であるが、非線形変換、典型的には$x$から$h$へのフィードフォワードニューラルネットによる変換であることが多い。同時に、線形ファクタモデルのように$h$から$x$の変換も学習する。  
したがって、線形ファクタモデルはデコーダのみを明示するが、auto-encoderはエンコーダも明示する。  
PCAのように線形ファクタモデルの中には、auto-endoer(線形の)に対応するようなものもあるが、  
その他については、encoderは$x$を生成しうる$h$を探索する推論のメカニズムの過程で暗に定義される。

<strong>
線形ファクタモデルは$h$-> $x$の説明性のみを考慮するが、  
auto-encodersは両方向を考慮する。
</strong>
 
 auto-encodersは長年研究されていたが、最近、注目されるようになった。
 
 auto-encodersは単に入力を出力にコピーしようとするフィードフォワードニューラルネットである。  
 記号は下記のように利用する。  
 ${\bf x}$：入力  
 $f$：endoder関数  
 ${\bf h}=f({\bf x})$：コード(内部表現)
 $g$：decoder関数  
 ${\bf r}=g({\bf h})=g(f({\bf x}))$：出力、reconstruction  
 $L({\bf r}, {\bf x})$：ロス関数
 
 ##補足
 よくある実装は下記の通り。  
 $
{\bf h}=\phi( W_1{\bf x} + b_1)\\
{\bf r}=\phi( W_2{\bf h} + b_2)
 $  
 tied weight制約を設けるのであれば、
 $W_1 = W_2$
 
目次  
1. Regularized Auto-Encoders  
2. Denoising Auto-Encoders  
3. Representational Power, Layer Size and Depth  
4. Reconstruction Distribution  
5. Linear Factor Models  
6. Probabilistic PCA and Factor Analysis  
7. Reconstruction Error as Log-Likelihood  
8. Sparse Representations  
9. Denoising Auto-Encoders  
10. Contractive Auto-Encoders  





## 1 Regularized Auto-Encoders  
<img src='files/auto-encoders_bottleneck.png' width="600px"/>
入力をそのまま出力するモデルでは、いい表現が得られないので、  
20世紀では、これを防ぐ為に${\bf h}$の次元数を${\bf x}$の次元数よりも少なくしていた。  
次元数を少なくするものを_undercomplete_といい、それ以外を_overcomplete_という。  
${\bf h}$の次元を少なくする**bottleneck**制約を利用しないovercompleteの場合、そのまま学習すると恒等関数が学習されてしまうが、  
別途制約を設けることで良い表現が学習できる。  

### 表現やその派生のスパース性について
高次元特徴量が抽出できたとしても，ほとんどの$h$が$0$の場合は有効なlocal dimensionaltyがより小さい可能性がある。  

* sparse coding  
${\bf h}=f({\bf x}) = \arg \min L(g({\bf h}), {\bf x})) + \lambda\Omega({\bf h})$  
とするモデル。  
例えば、多くが0か0に近い値となるスパース性を達成したいのであれば、  
L1正則化$\Omega({\bf h}) = |{\bf h}|$を利用する。   

* PSD(predictive sparse decomposition)  
後述。

* sparse auto-encoders  
他の正則化項として、Student-t penalty、KL-divergence penaltyなどがある。  

* contractive autoencoders  
$\Omega({\bf h})$の制約を設ける。
$f$の出力(${\bf h}$)が収縮するので、contractive autoencodersという。


### ノイズや欠損へのロバスト性について
auto-encoderは欠損やノイズのないものとして入力の完全な再構築を行おうとするものであり、   
ノイズや欠損があると恒等関数を学習することはできない。  
したがって、こういった場合に最適な再構築を行う為には、データ分布の構造を捉える必要があり、  
こうしたauto-encodersをdenoising auto-encodersとよぶ。  

## 2 Denoising Auto-Encoders  
### denoising auto-encodersとcontractive auto-encodersには強いつながりがある。 
(説明が中途半端でロジックが通っていない、、)  
ガウス分布に従う小さなノイズを入力に加えるDAEのreconstruction errorは、   
reconstruction関数におけるcontractive penaltyと等価である。   
言い換えると、${\bf x}$と${\bf x}+{\bf \epsilon}$を同じ${\bf x}$に返す為には、全方向の${\bf \epsilon}$の変化に対して敏感でないことが期待される。  

### 表現のPriorによる制約
表現に制約を加えるという概念を一般化する方法として、  
$−logP({\bf h})$を加えるという方法がある。
単一の分布に従う表現か、元々のデータよりも単純な表現を学習することがモチベーションであり、  
${\bf h}$をよりシンプルにする(少ないパタンで表現される)>>$−logP({\bf h})$を下げればいい、という考えによる方法である。  
変分auto-encodersでは、ちゃんと、こういったのが組み込まれているらしい。

## 3. Representational Power, Layer Size and Depth  
### 層の数
これまでの議論には、多層化できないという話はでていないが、単層のautoencodersの事例が多い。  
### 理由1：Universal Approximation Theory
十分大きな数のUnit数をもつ単層の隠れ層のニューラルネットは指定の精度でどんな関数も近似できる。  
これは、Overcompleteを正当化させる理由の一つ。  
PCAはundercompleteの事例の一つ(ここで、取り上げる必要あったのか？)。

### 理由2：Layerwiseに学習させる方法がメジャー
deep auto-encodersでは、Layerwiseに学習し重ねてinitializationする方法でうまくいっており、  
議論の中心が単層の学習方法についてであることが多いため。


## 4. Reconstruction Distribution  
これまでの部分は、RMSEによるreconstruction errorを念頭に置いていたが、  
必ずしもMSEが適切というわけではない。  
例えば、  
${\bf x}$が離散値の場合や、  
$P({\bf x}|{\bf h})$がガウス分で近似できない場合など。  
通常では、  
${\bf x}$が上下界のない実数値の場合はガウス分布を仮定し、  
${\bf x}$が2値の場合はベルヌーイ分布を仮定し、  
モデル化することが多いが、  
他の混合分布を仮定しても良い。

<img src='files/reconstruction_distribution.png' width="600px"/>

decoding function  $g({\bf h})$の概念はdecoding distribution  $P({\bf x}|{\bf h})$に一般化できる。  
同様に、  
encoding function  $f({\bf x})$の概念はencoding distribution  $P({\bf h}|{\bf x})$に一般化できる。   
こうすることで、潜在変数のように考えることで、${\bf h}$の段階でノイズが含まれている事実を捉えることができる。

確率的encoderや確率的decoderがRBMにはあるが、詳細は20章にて、

## 5. Linear Factor Models  
$
{\bf h} \sim P({\bf h})\\
{\bf x}  = {\bf W} {\bf h}  + {\bf b}  + noise
$  
ノイズはガウス分布に従う対角のものを使うことが多い。   

<img src='files/linear_factor_models.png' width="600px"/>


## 6. Probabilistic PCA and Factor Analysis  
確率的PCAやfactor analysis、他のlinear factor モデルは、上記の数式の特別な事例。  

factor analysisでは、  
${\bf h} \sim N(0,{\bf I})$  
のように、潜在変数のpriorは単に分散1のガウス分布とされ、
$x$は互いに独立であり、  
ノイズ対角分散ガウス分布から生成されると仮定されている。  

潜在変数の役割は、異なる観測データにおける$x$の違いを捉えることである。  
${\bf x} $は単にガウス分布のランダム変数であることが簡単に示すことができる。  
${\bf x}  \sim N({\bf b} , {\bf WW}  +\psi)$  
ただし、$\psi=diag ({\bf \sigma})$のような共分散行列。  

確率的PCAについてでは、

共分散が$\sigma$  が全変数で同じだと仮定して、  
${\bf x}  \sim N({\bf b} , {\bf WW}  +\sigma{\bf I})$    
とするか、全く等価に  
${\bf x}  = {\bf Wh}  + {\bf b} + \sigma z$    
ただし、$z  \sim N(0, {\bf I})$    
のように表現するか。  
$\sigma \rightarrow  0$の時、確率的PCAはPCAになる。

## 7. Reconstruction Error as Log-Likelihood  
誤差関数は二乗誤差である必要はない．  
Negative Log likelihoodもつかえる。  
ロス関数は下記の通り。
$L = − \log P ( {\bf x}   |   {\bf   h  } )$  
ロス関数は、基本的にデータ${\bf x}$の特徴に合わせる。  
実数の上下界がないデータであれば二乗誤差で、  
2値であればベルヌーイ分布を仮定してcross-entropyを選択するのが妥当。 




## 8. Sparse Representations  
### Sparse coding
auto-encoderじゃない。  
${\bf x}$を説明する${\bf h}$をどうやってつくるかの話が中心？？。   
${\bf h}=f({\bf x}) = \arg \min L(g({\bf h}), {\bf x})) + \lambda\Omega({\bf h})$  

### Sparse auto-encoders
$L = - \log P( {\bf x} |g({\bf h})) + \lambda\Omega({\bf h})$  
例えば、L1正則化して、0か0に近い値を増やしたい場合は、  
$\Omega({\bf h})= \lambda |{\bf h}|$  

ターゲットをしてしたい場合(例えば$\rho=0.05$)は、   
$\Omega({\bf h})= \rho \log h + (1 - \rho) \log (1-h)$  

### Predictive Sparse Decomposition
物体認識の領域で利用されるpretrainng手法らしい。  
TODO：説明加筆。  
双方向の誤差を同時に最小化しつつ、$h$のノルムを小さくするように学習。  
$L=argmin(   \|{\bf x}−g({\bf h})\| +\lambda |{\bf h}| +\gamma \|{\bf h}−f({\bf x})\|)$  
最初の2項はsparse codingと同じ。

## 9. Denoising Auto-Encoders  
<img src='files/dae1.png' width="600px"/>
1. 訓練データから${\bf x}$をサンプリング。
2. ${\bf x}$にノイズを加えた${\bf \tilde{x}}$を  
条件付き分布$C(\bf {\tilde{x}}|{\bf x})$よりサンプリング。
3. $({\bf x}, {\bf \tilde{x}})$を訓練データとして、  
auto-encoder reconstruction分布
$P(\bf {\bf x}|{\tilde{x}}) = P(\bf {x}|g({\bf h}))$（ただし、${\bf h} = f({\bf \tilde{x}})$）
を推定。

DAEは平均二乗誤差$\|g(f({\bf \tilde{x}})) - {\bf x}\|$を最小化するように学習するので、   
多様体に対して、直交するような形で学習する。  
学習の基準がDAEに勾配場を推定するベクトル場$(g(f({\bf \tilde{x}})) - {\bf x})$の学習を行わせている。
<img src='files/dae2.png' width="600px"/>

## 10. Contractive Auto-Encoders  
コードに$\Omega({\bf h})$の制約を設ける。
$f$の出力${\bf h}$が収縮するので、contractive autoencodersという。

$$\Omega({\bf h}) = \frac{\delta f({\bf x})}{\delta {\bf x}}$$
ヤコビアンの行列?の2乗ノルムの和など。  
データの復元に必要でないほとんどの次元で小さな値を取るように学習する。

<img src='files/CAE1.png' width="600px"/>
利点：多様体に直交する方向に対してより普遍性の高い表現を学習できる。  
理由：他のautoencodersより少ない次元に表現の感度を集中させようとする為、
<img src='files/CAE2.png' width="600px"/>
両方とも、ローカルな接線はとれているが、
local PCAはよりグローバルないい接線方向の抽出ができていない。  

practical issue：
* deep CAEを学習させる場合はlayer-wiseに学習させる。  
* tied weightつけないと、contraction penaltyがききにくい。
